In [1]:
from esios.archives import preprocessing

In [2]:
import glob
import pandas as pd

sheet_name = 'I90DIA08'
files = glob.glob("data/I90DIA/*/*.xls")
files.sort()

dfs = []
for path in files:
    df = preprocessing.process_excel_file(path, sheet_name)
    dfs.append(df)

df = pd.concat(dfs).sort_values('datetime')
df

,Redespacho,Tipo,Sentido,Unidad de Programación,Tipo Oferta,Tipo cálculo,Tipo Restricción,Signo de Energía,datetime,value
0,Restricciones Técnicas,UPOTROR,Subir,BES4,1.0,No aplica,RTT,Positivo,2020-01-01 00:00:00+01:00,NaN
11,Indisponibilidad,INDIS,Bajar,TIGBCL,10.0,No aplica,N/A,Negativo,2020-01-01 00:00:00+01:00,NaN
...,...,...,...,...,...,...,...,...,...,...
3236,Restricciones Técnicas,UPOTROR,Subir,CTGN1,1.0,No aplica,RSI,Positivo,2024-09-22 23:45:00+02:00,NaN
3263,Otros,ACCBAL,Bajar,ACBFRAE,5.0,No aplica,N/A,Negativo,2024-09-22 23:45:00+02:00,NaN


In [3]:
df.to_parquet('i90dia08.parquet')

,,,,,,,,datetime,value
Redespacho,Tipo,Sentido,Unidad de Programación,Tipo Oferta,Tipo cálculo,,,,
Restricciones Técnicas,UPOTROR,Subir,BES4,1.0,No aplica,RTT,Positivo,2020-01-01 00:00:00+01:00,NaN
Indisponibilidad,INDIS,Bajar,TIGBCL,10.0,No aplica,N/A,Negativo,2020-01-01 00:00:00+01:00,NaN
...,...,...,...,...,...,...,...,...,...
Restricciones Técnicas,UPOTROR,Subir,SAGU1,1.0,Complejo,Positivo,RSI,2024-09-22 23:45:00+02:00,38.5
Desvíos,DESCOM2,Bajar,FGASNE2,10.0,No aplica,Negativo,N/A,2024-09-22 23:45:00+02:00,NaN


In [10]:
path

'data/I90DIA/I90DIA_20200709/I90DIA_20200709.xls'

In [6]:
from pathlib import Path
import pandas as pd

In [18]:
import io
from typing import Iterator, IO

In [25]:
import python_calamine

def iter_excel_calamine(file: IO[bytes]) -> Iterator[dict[str, object]]:
    workbook = python_calamine.CalamineWorkbook.from_filelike(file)  # type: ignore[arg-type]
    rows = iter(workbook.get_sheet_by_index(0).to_python())
    headers = list(map(str, next(rows)))
    for row in rows:
        yield dict(zip(headers, row))

In [38]:
import python_calamine
from typing import Union, Iterator, IO

def iter_excel_calamine(file: Union[str, IO[bytes]], sheet_name: str) -> Iterator[dict[str, object]]:
    """
    Iterate over rows of a specific sheet in an Excel file using python_calamine.

    Args:
        file (Union[str, IO[bytes]]): File path or file-like object containing the Excel file.
        sheet_name (str): Name of the sheet to read.

    Yields:
        Iterator[dict[str, object]]: Rows as dictionaries with headers as keys.
    """
    # If file is a path, open it in binary mode
    if isinstance(file, str):
        with open(file, "rb") as f:
            workbook = python_calamine.CalamineWorkbook.from_filelike(f)
            sheet = workbook.get_sheet_by_name(sheet_name)
            rows = iter(sheet.to_python())
            headers = list(map(str, next(rows)))
            for row in rows:
                yield dict(zip(headers, row))

In [81]:
file = files[0]
file

'data/I90DIA/I90DIA_20221014/I90DIA_20221014.xls'

In [92]:
with open(file, "rb") as f:
    workbook = python_calamine.CalamineWorkbook.from_filelike(f)
    sheet = workbook.get_sheet_by_name(sheet_name)
    rows = iter(sheet.to_python())

In [93]:
df = pd.DataFrame(rows)
df

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,,,,,,,,,Indicadores,MWh,...,,,,,,,,,,
1,,,,,,,,,Dia,2022-10-14,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,Indisponibilidad,INDIS,Bajar,TPGUIZ,10.0,No aplica,N/A,Negativo,,-382.5,...,,,,,,,,,,
25,Otros,ACCBAL,Subir,ACBFRAI,5.0,No aplica,N/A,Positivo,,323.0,...,,,,,,,,,,


In [105]:
import pandas as pd
import python_calamine
from typing import IO, Union


def read_excel_with_detected_headers(file: Union[str, IO[bytes]], sheet_name: str) -> pd.DataFrame:
    """
    Read an Excel file using python_calamine, detecting the first non-empty row as headers.

    Args:
        file (Union[str, IO[bytes]]): Path to the Excel file or file-like object.
        sheet_name (str): Name of the sheet to read.

    Returns:
        pd.DataFrame: Transformed data as a Pandas DataFrame.
    """
    if isinstance(file, str):
        with open(file, "rb") as f:
            return _process_workbook(f, sheet_name)
    else:
        return _process_workbook(file, sheet_name)


def _process_workbook(file: IO[bytes], sheet_name: str) -> pd.DataFrame:
    workbook = python_calamine.CalamineWorkbook.from_filelike(file)
    sheet = workbook.get_sheet_by_name(sheet_name)
    rows = list(sheet.to_python())  # Convert rows to a list for processing

    # Detect the first non-empty row as headers
    headers = []
    data_start_row = 0
    for i, row in enumerate(rows):
        if any(cell is not None and str(cell).strip() for cell in row):  # Check for non-empty row
            headers.append(row)
            data_start_row = i + 1  # Data starts from the next row
            break

    if headers is None:
        raise ValueError("No headers found in the sheet.")

    # Create DataFrame with detected headers and data
    data = rows[data_start_row:]
    df = pd.DataFrame(data, columns=headers)

    return df

In [195]:
file = Path(files[0])

In [178]:
j

0

In [171]:
rows[i-2]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Dia',
 datetime.date(2022, 10, 14),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [164]:
row[:j]

[]

In [152]:
df.head(10).style

\
                                                                       
0               Redespacho     Tipo  Sentido  Unidad de Programación   
1   Restricciones Técnicas  UPOTROR    Subir                  ARCOS1   
..                     ...      ...      ...                     ...   
22        Indisponibilidad    INDIS    Bajar                  TPGUIZ   
23                   Otros   ACCBAL    Subir                 ACBFRAI   

                                                                   \
                                                                    
0   Tipo Oferta  Tipo cálculo  Tipo Restricción  Signo de Energía   
1           1.0      Complejo               RSI          Positivo   
..          ...           ...               ...               ...   
22         10.0     No aplica               N/A          Negativo   
23          5.0     No aplica               N/A          Positivo   

               Indicadores        MWh  ...                                \
                       Dia 2022-10-14  ...                                 
0   Cuarto de Hora del dia      Total  ...  87.0  88.0  89.0  90.0  91.0   
1                               468.0  ...  39.0  39.0                     
..                     ...        ...  ...   ...   ...   ...   ...   ...   
22                             -382.5  ...                                 
23                              323.0  ...                                 

                                  
                                  
0   92.0  93.0  94.0  95.0  96.0  
1                                 
..   ...   ...   ...   ...   ...  
22                                
23                                

[24 rows x 106 columns]

In [133]:
cell

'ACCBAL'

In [135]:
rows

[['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'Indicadores',
  'MWh',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'Dia',
  datetime.date(2022, 10, 14),
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '

In [127]:
pd.DataFrame(rows).head(10).style

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
0,,,,,,,,,Indicadores,MWh,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,Dia,2022-10-14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Redespacho,Tipo,Sentido,Unidad de Programación,Tipo Oferta,Tipo cálculo,Tipo Restricción,Signo de Energía,Cuarto de Hora del dia,Total,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000,32.000000,33.000000,34.000000,35.000000,36.000000,37.000000,38.000000,39.000000,40.000000,41.000000,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000,52.000000,53.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.000000,60.000000,61.000000,62.000000,63.000000,64.000000,65.000000,66.000000,67.000000,68.000000,69.000000,70.000000,71.000000,72.000000,73.000000,74.000000,75.000000,76.000000,77.000000,78.000000,79.000000,80.000000,81.000000,82.000000,83.000000,84.000000,85.000000,86.000000,87.000000,88.000000,89.000000,90.000000,91.000000,92.000000,93.000000,94.000000,95.000000,96.000000
3,Restricciones Técnicas,UPOTROR,Subir,ARCOS1,1.000000,Complejo,RSI,Positivo,,468.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,,,,,,,,
4,Restricciones Técnicas,UPOTROR,Subir,ARRU1,1.000000,Complejo,RSI,Positivo,,12.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1.500000,1.500000,1.500000,1.500000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1.500000,1.500000,1.500000,1.500000
5,Restricciones Técnicas,UPOTROR,Subir,CTN3,1.000000,No aplica,RSI,Positivo,,2480.100000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,53.475000,53.475000,53.475000,53.475000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,34.725000,34.725000,34.725000,34.725000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,75.975000,,,,
6,Restricciones Técnicas,UPOTROR,Subir,CTNU,1.000000,Complejo,RSI,Positivo,,1800.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
7,Restricciones Técnicas,UPOTROR,Subir,ECT2,1.000000,Complejo,RSI,Positivo,,222.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,18.500000,,,,,,,,
8,Restricciones Técnicas,UPOTROR,Subir,ESC6,1.000000,Complejo,RSI,Positivo,,1564.500000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,78.225000,,,,,,,,
9,Restricciones Técnicas,UPOTROR,Subir,ESCCC1,1.000000,Complejo,RSI,Positivo,,1240.000000,,,,,,,,,,,,,,,,,

In [122]:
cell

'Otros'

In [108]:
df = read_excel_with_detected_headers(file, sheet_name)
df

,,,,,,,,,Indicadores,MWh,...,,,,,,,,,,
0,,,,,,,,,Dia,2022-10-14,...,,,,,,,,,,
1,Redespacho,Tipo,Sentido,Unidad de Programación,Tipo Oferta,Tipo cálculo,Tipo Restricción,Signo de Energía,Cuarto de Hora del dia,Total,...,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Indisponibilidad,INDIS,Bajar,TPGUIZ,10.0,No aplica,N/A,Negativo,,-382.5,...,,,,,,,,,,
24,Otros,ACCBAL,Subir,ACBFRAI,5.0,No aplica,N/A,Positivo,,323.0,...,,,,,,,,,,


In [98]:
df

,,,,,,,,,Indicadores,MWh,...,,,,,,,,,,
0,,,,,,,,,Dia,2022-10-14,...,,,,,,,,,,
1,Redespacho,Tipo,Sentido,Unidad de Programación,Tipo Oferta,Tipo cálculo,Tipo Restricción,Signo de Energía,Cuarto de Hora del dia,Total,...,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Indisponibilidad,INDIS,Bajar,TPGUIZ,10.0,No aplica,N/A,Negativo,,-382.5,...,,,,,,,,,,
24,Otros,ACCBAL,Subir,ACBFRAI,5.0,No aplica,N/A,Positivo,,323.0,...,,,,,,,,,,


In [22]:
(path)

In [24]:
next(data)

StopIteration: 

In [23]:
pd.DataFrame(data)

TypeError: Object does not have a .read() method.

In [7]:
if not isinstance(path, Path):
    path = Path(path)

# Read the entire sheet once
full_sheet = pd.read_excel(path, sheet_name=sheet_name, skiprows=0, header=None)

# Determine the position of "Indicadores"
position_row = full_sheet.iloc[0]
position = position_row.tolist().index("Indicadores")


In [13]:
from xlsx2csv import Xlsx2csv
from io import StringIO
import pandas as pd


def read_excel(path: str, sheet_name: str, params: dict) -> pd.DataFrame:
    buffer = StringIO()
    Xlsx2csv(path, outputencoding="utf-8", sheet_name=sheet_name).convert(buffer)
    buffer.seek(0)
    df = pd.read_csv(buffer, **params)
    return df

In [16]:
path

PosixPath('data/I90DIA/I90DIA_20221014/I90DIA_20221014.xls')

In [15]:
df = read_excel(path, sheet_name, params=dict(
    header=[0,1,2],
))

df

InvalidXlsxFileException: Invalid xlsx file: data/I90DIA/I90DIA_20221014/I90DIA_20221014.xls

In [10]:
import dask

# note we are wrapping in delayed only the function, not the arguments
delayeds = [dask.delayed(pd.read_excel)(i, sheet_name=sheet_name, skiprows=2) for i in files[:10]]

# the line below launches actual computations
results = dask.compute(delayeds)

In [9]:
results

([                Redespacho     Tipo Sentido Unidad de Programación  \
  0   Restricciones Técnicas  UPOTROR   Subir                 ARCOS1   
  1   Restricciones Técnicas  UPOTROR   Subir                  ARRU1   
  ..                     ...      ...     ...                    ...   
  21        Indisponibilidad    INDIS   Bajar                 TPGUIZ   
  22                   Otros   ACCBAL   Subir                ACBFRAI   
  
      Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía  \
  0             1     Complejo              RSI         Positivo   
  1             1     Complejo              RSI         Positivo   
  ..          ...          ...              ...              ...   
  21           10    No aplica              NaN         Negativo   
  22            5    No aplica              NaN         Positivo   
  
      Cuarto de Hora del dia  Total  ...    87    88  89  90  91  92   93   94  \
  0                      NaN  468.0  ...  39.0  39.0 NaN NaN NaN NaN  Na

In [11]:
import dask.dataframe as dd

df = dd.read_excel(path, sheet_name=sheet_name)
# Process data as Dask DataFrame
df.compute()  # Converts to pandas DataFrame

AttributeError: module 'dask.dataframe' has no attribute 'read_excel'

In [15]:
import pandas as pd

In [16]:
excel_file = pd.ExcelFile(path)
sheet_data = excel_file.parse(sheet_name, skiprows=0, nrows=1)
position = sheet_data.columns.get_loc("Indicadores")

df = excel_file.parse(sheet_name, index_col=list(range(position)), skiprows=2).iloc[:, 2:]

In [17]:
df

1   \
Redespacho             Tipo    Sentido Unidad de Programación Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía       
Restricciones Técnicas UPOTROR Subir   ARCOS1                 1           Complejo     RSI              Positivo         NaN   
                                       ARRU1                  1           Complejo     RSI              Positivo         NaN   
...                                                                                                                       ..   
Indisponibilidad       INDIS   Bajar   TPGUIZ                 10          No aplica    NaN              Negativo         NaN   
Otros                  ACCBAL  Subir   ACBFRAI                5           No aplica    NaN              Positivo         NaN   

                                                                                                                          2   \
Redespacho             Tipo    Sentido Unidad de Programación Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía       
Restricciones Técnicas UPOTROR Subir   ARCOS1                 1           Complejo     RSI              Positivo         NaN   
                                       ARRU1                  1           Complejo     RSI              Positivo         NaN   
...                                                                                                                       ..   
Indisponibilidad       INDIS   Bajar   TPGUIZ                 10          No aplica    NaN              Negativo         NaN   
Otros                  ACCBAL  Subir   ACBFRAI                5           No aplica    NaN              Positivo         NaN   

                                                                                                                          3   \
Redespacho             Tipo    Sentido Unidad de Programación Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía       
Restricciones Técnicas UPOTROR Subir   ARCOS1                 1           Complejo     RSI              Positivo         NaN   
                                       ARRU1                  1           Complejo     RSI              Positivo         NaN   
...                                                                                                                       ..   
Indisponibilidad       INDIS   Bajar   TPGUIZ                 10          No aplica    NaN              Negativo         NaN   
Otros                  ACCBAL  Subir   ACBFRAI                5           No aplica    NaN              Positivo         NaN   

                                                                                                                          4   \
Redespacho             Tipo    Sentido Unidad de Programación Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía       
Restricciones Técnicas UPOTROR Subir   ARCOS1                 1           Complejo     RSI              Positivo         NaN   
                                       ARRU1                  1           Complejo     RSI              Positivo         NaN   
...                                                                                                                       ..   
Indisponibilidad       INDIS   Bajar   TPGUIZ                 10          No aplica    NaN              Negativo         NaN   
Otros                  ACCBAL  Subir   ACBFRAI                5           No aplica    NaN              Positivo         NaN   

                                                                                                                          5   \
Redespacho             Tipo    Sentido Unidad de Programación Tipo Oferta Tipo cálculo Tipo Restricción Signo de Energía       
Restricciones Técnicas UPOTROR Subir   ARCOS1                 1           Complejo     RSI              Positivo         NaN   
                                       ARRU1                  1           Complejo     RSI              Positivo         NaN   
...                   

In [7]:

dfs = []
for file in files:
    df = preprocessing.process_excel_file(file, sheet_name='I90DIA08')
    dfs.append(df)

KeyError: 'Indicadores'